<a href="https://colab.research.google.com/github/Sahilsingh0808/US-Debt-Structures/blob/main/US_states.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Sets

In [4]:
import pickle
from os import walk
import pandas as pd
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import math
import tabula

Declarations

In [5]:
tables=[]
year=2020 #change year here
state="alaska" #change state here
path =""
dataCopy=pd.DataFrame()
finalData=pd.DataFrame(columns=['Due Year','Principal','Interest','Swap Net Payment','Total'])

***Functions***

Identify Type of Table

In [ ]:
def identify_table(tabdata, yr, threshold):

    indexcol = tabdata.iloc[:,0]    
    valuelist  = [it for it  in indexcol.to_list() if it != ""]
    count = 0
    for ele in valuelist:
        try:
            newele = int(ele)
            #print(newele)
            if newele in list(range(yr,yr+30)):
                count += 1
        except:
            pass
    
    if len(valuelist) == 1:
        freq = count / (len(valuelist))
    else:
        freq = count / (len(valuelist) - 1)
    
    d_mattable=freq > threshold
    if d_mattable:
        concatStr=""
        rowC=tabdata.shape[0]
        colC=len(tabdata.columns)
        for k in range(0,rowC):
            row=tabdata.iloc[k].to_list()
            for l in range(0,len(row)):
                concatStr+=str(row[l])+" "
        findPrincipal=re.search(r"principal",concatStr,re.IGNORECASE)
        findInterest=re.search(r"interest",concatStr,re.IGNORECASE)
        if findPrincipal or findInterest:
            return True
        elif not findPrincipal and not findInterest:
            return False
    else:
        return False

Table ID

In [ ]:
def get_table_id(tab):
    table_id=""
    table=tab[:-4]
    for i in range(0,len(table)-3):
        if table[i]=="_" and table[i+1]=="i" and table[i+2]=="d" and table[i+3]=='_':
            table_id=table[i+1:len(table)-2]
            break
    return table_id

Repair df index

In [ ]:
def repair_dfindex(tabdata):
    if tabdata.iloc[-1,0] == "":
        tabdata.iloc[-1,0] = "total"
    else:
        pass
    
    
    for idx,row in tabdata.iterrows():
        
        indexvalue = row.iloc[0]
        indexvalue = indexvalue.strip().lower()
        #print(indexvalue)

        brokenhyphen = re.search("^(\d{4})(\s*)(\d{4})$",indexvalue)

        if brokenhyphen:
            #print(brokenhyphen)
            newvalue = f"{brokenhyphen.group(1)}-{brokenhyphen.group(3)}"
            tabdata.loc[idx,  tabdata.columns[0]  ] = newvalue

    return tabdata

Get Headings

In [ ]:
def get_headings(data):
    headings=[]
    data_list=data.values.tolist()
    for i in range(0,len(data_list)):
        for j in range(0,len(data_list[i])):
            if "Year" in str(data_list[i][j]) or "year" in str(data_list[i][j]):
                if i==0:
                    headings=data_list[i]
                else:
                    x=(data_list[i-1])
                    y=(data_list[i])
                    for j in range(0,min(len(x),len(y))):
                        if y[j]!="" and y[j]!="\n":
                            headings.append(x[j]+" "+y[j])
                   
                break
    return headings

Check Float

In [ ]:
def check_float(x):
    try:
        ff = float(x)
        return True 
    except ValueError:
        return False

Check First Row

In [ ]:
def check_firstrow(x):
    # do some cleaning
    x = x.replace("$","").replace("S","")
    x = x.strip().lower()
    
    d_empty = x == ""
    d_numeric = check_float(x)
    
    if  d_empty or not d_numeric:
        return True
    else:
        return False

Adjust Colheader

In [ ]:
def adjust_colheaderformat(x):
    x = str(x)
    x = x.strip().lower()
    x = x.replace("-","").replace(" ","_").strip('\''',').replace(',','')
    x = x.replace(r"(","")
    x = x.replace(r")","")
    x = x.replace(r"__","_").replace(r"__","_")
    return x

Final Adj Colheader

In [ ]:
def finaladj_colheader(x):
    x = re.sub('^\_', '', x)
    x = re.sub('\_$', '', x)
    return x

Get Colheader

In [ ]:
def get_colheader(tabdata, camelot):
    if camelot:
        dflist = tabdata.values.tolist()
    else:
        dflist = [list(tabdata.columns)] + tabdata.values.tolist()
    
    rowindex = 0
    
    while check_firstrow(dflist[rowindex][0]) and rowindex < len(dflist):
        rowindex += 1
    
    if rowindex > 0:
        # print("Multicolumn header")
        newcolsdict = {0:""}
        for i in range(1,len(tabdata.columns)):
            aux_header = "_".join( [ adjust_colheaderformat(dflist[rowidx][i]) for rowidx in range(rowindex) ] ) 
            
            header =  finaladj_colheader(aux_header )
            
            newcolsdict.update({i:  header })    
        
    else:
        newcolsdict = dict([[idx, adjust_colheaderformat(col) ] for idx,col in enumerate(tabdata.columns)])
        
    return newcolsdict

Delete Empty Rows

In [ ]:
def delete_empty_rows(data):
    data =data.dropna(axis=0, how='all', inplace=True) 
    return data

Delete Empty Columns

In [ ]:
def delete_empty_columns(data,headings):
    # rowC=data.shape[0]
    # colC=len(data.columns)
    # for i in range(0,colC):
    #     for j in range(0,rowC):
    #         try:
    #             print(str(data[i][j]))
    #         except:
    #             pass
    cols = data.columns.tolist()
    for i in range(1,len(cols)):
        try:
            column=data[cols[i]]
            # print((column))
            sum=0.0
            for j in range(0,len(column)):
                try:
                    cell=float(column[j])
                    sum+=cell
                except:
                    pass
            if sum==0.0:
                # print("FOUND EMPTY COLUMN")
                # print(i)
                if "principal" in str(headings[i]).lower() or "interest" in str(headings[i]).lower() or "total" in str(headings[i]).lower() or "swap" in str(headings[i]).lower() or "net" in str(headings[i]).lower():
                    pass
                else:                    
                    data=data.drop(cols[i],axis=1)
        except:
            pass
        
        # for j in range(0,len(column)):
        #     sum+=float(column[j])
        # if sum==0.0:
        #     data=data.drop(columns=cols[i])
            # if type(column[j])!=float and type(column[j])!=int:
            #     print(column[j])
    return data

Check String

In [ ]:
def checkString(str1):
   
    # initializing flag variable
    flag_l = False
    flag_n = False
     
    # checking for letter and numbers in
    # given string
    for i in str1:
       
        # if string has letter
        if i.isalpha():
            flag_l = True
 
        # if string has number
        if i.isdigit():
            flag_n = True
     
    # returning and of flag
    # for checking required condition
    return flag_l or flag_n

Delete Empty Rows Values

In [ ]:
def delete_empty_rows_values(data):
    data_list=data.values.tolist()
    for i in range(0,len(data_list)):
        if data_list[i]==[]:
            data=data.drop(data.index[i])
        elif data_list[i]==[np.nan]:
            data=data.drop(data.index[i])
        cc=0
        for j in range(0,len(data_list[i])):

            # print(data_list[i][j])
            if checkString(str(data_list[i][j]))==False or data_list[i][j]=="0.0" or data_list[i][j]=="0":
                cc+=1
            # if checkString(str(data_list[i][j]))==False:
            #     data[i][j]="0"
        
        if cc==(len(data_list[i])-1):
            data=data.drop(data.index[i])
    return data


Correct Numbers Table

In [ ]:
def correct_numbers_table(data):
    rowC=data.shape[0]
    colC=len(data.columns)
    for i in range(1,colC):
        for j in range(0,rowC):
            cell=str(data[i][j])
            newNum=""
            for k in cell:
                if k.isdigit()==True or k=='.':
                    newNum+=k
            data[i][j]=newNum
    return data

Interpolate

In [ ]:
def interpolate(data):
    rowC=data.shape[0]
    colC=len(data.columns)
    data_list = data.values.tolist()
    
    
    for i in range(0,len(data_list[0])):
        for j in range(0,len(data_list)):
            #changing null values to 0
            if data_list[j][i]=='':
                data_list[j][i]=0.0
            #converting string values to numbers
            try:
                if i>0:
                    data_list[j][i]=float(data_list[j][i])
            except:
                print("ERROR "+data_list[j][i])
    data=pd.DataFrame(data_list)
    insertL=[[]]
    for i in range(0,len(data_list)):
        # print(data_list[i])
        year=str(data_list[i][0])
        if(len(year)>4):
            years=year.split("-")
            starting_year=int(years[0])
            ending_year=int(years[1])
            diff=int(ending_year-starting_year+1)
            insert_list=[]
            
            for j in range(1,len(data_list[0])):
                cell=data_list[i][j]
                # print(cell)
                insert_list.append(cell/diff)
            # print(insert_list)

            for j in range(0,diff):
                yr=str(starting_year+j)
                insert_list.insert(0,yr)
                # print(insert_list)
                data.loc[len(data)] = insert_list
                # insertL.append(insert_list)
                insert_list.pop(0)
            # print(insertL)
            insert_list.clear()

    data_list1=data.values.tolist()
    for i in range(0,len(data_list1)):
    # print(data_list[i])
        year=str(data_list1[i][0])
        if(len(year)>4):
            # print(year)
            data=data.drop(index=i)      

    # print(data_list)

    # for i in range(0,rowC):
    #     year=str(data[0][i])
    #     print(year)
    #     if(len(year)>4):
    #         years=year.split("-")
    #         starting_year=int(years[0])
    #         ending_year=int(years[1])
    #         diff=ending_year-starting_year+1
    #         insert_list=[]
    #         for j in range(1,colC):
    #             # if re.match(r'^-?\d+(?:\.\d+)$', data[i][j]) is None:
    #             #     print ("Not float")
    #             # else:
    #             #     insert_list.append(data[i][j]/diff)
    #             try:
    #                 print((data[i][j]))
    #                 insert_list.append(int(data[i][j])/diff)
    #             except:
    #                 pass
    #         print(insert_list)

    return data


Delete Empty Columns Extra

In [ ]:
def delete_empty_columns_extra(data,extra):
    cc=0
    # rowC=data.shape[0]
    # colC=len(data.columns)
    # for i in range(0,colC):
    #     for j in range(0,rowC):
    #         try:
    #             print(str(data[i][j]))
    #         except:
    #             pass
    cols = data.columns.tolist()
    for i in range(1,len(cols)):
        try:
            column=data[cols[i]]
            # print((column))
            sum=0.0
            for j in range(0,len(column)):
                try:
                    cell=float(column[j])
                    sum+=cell
                except:
                    pass
            if sum==0.0 and cc<extra:
                # print("FOUND EMPTY COLUMN")
                # print(i)
                cc+=1             
                data=data.drop(cols[i],axis=1)
        except:
            pass
        
        # for j in range(0,len(column)):
        #     sum+=float(column[j])
        # if sum==0.0:
        #     data=data.drop(columns=cols[i])
            # if type(column[j])!=float and type(column[j])!=int:
            #     print(column[j])
    return data

Format Headings

In [ ]:
def format_headings(headings):
    
    for i in range(0,len(headings)):
        headings[i]=headings[i].replace("_"," ")
        headings[i]=headings[i].replace("/"," ")
        headings[i]=headings[i].replace("\\"," ")
        headings[i].strip()
        
        headings[i]=headings[i].lower()
        if "principal" in headings[i]:
            headings[i]="Principal"
        elif "interest" in headings[i]:
            headings[i]="Interest"
        elif "total" in headings[i]:
            headings[i]="Total"
        elif "swap net" in headings[i]:
            headings[i]="Swap Net Payment"
        elif "operating leases" in headings[i]:
            headings[i]="Total"
            

    headings[0]="Due Year"

    return headings

Filenames

In [7]:
filenames = next(walk(r'{path}'), (None, None, []))[2] 
print(filenames)

[]


Restricting to a particular state

In [ ]:
for i in range(0,len(filenames)):
    try:
        if state in str(filenames[i]):
            continue
        else:
            filenames[i]=None
    except:
        pass
filenames = list(filter(None, filenames))

Final Algorithm

In [ ]:
for i in range(len(filenames)):
    with open(f"C:\\Users\\sahil\\Dropbox\\MigrationData\\rawtablesnew\\{filenames[i]}", 'rb') as f:
        data = pickle.load(f)
        table_id=get_table_id(filenames[i])
        tables.append(data)
        d_mattable=identify_table(data, year, threshold = .2)
        if d_mattable:

            #repair index
            data=repair_dfindex(data)

            dataCopy=data

            #column headers
            headings=get_headings(data)
            
            newcolsdict = get_colheader(data,camelot=True)

            # print(data)
            print(newcolsdict)
            print(table_id)
            
            #print whole table
            with pd.option_context('display.max_rows', None, 'display.max_columns', None):

                #delete a completely empty row
                delete_empty_rows(data)  
                rowC=data.shape[0]
                colC=len(data.columns)

                data_list=data.values.tolist()

                #ignoring the starting useless rows of the dataframe
                for i in range(0,len(data_list)):
                    for j in range(0,len(data_list[i])):
                        cell=str(data_list[i][j])
                        if "Year" in cell or "year" in cell:
                            data=(data.iloc[i+1:,])
                            # print(data)
                            break

                data=repair_dfindex(data)
                data=data.reset_index(drop=True)

                #ignoring the ending useless rows of the dataframe
                data_list=data.values.tolist()
                for i in range(0,len(data_list)):
                    for j in range(0,len(data_list[i])):
                        cell=str(data_list[i][j])
                        if "Total" in cell or "total" in cell:
                            data=(data.iloc[:i,])
                            break

                data=delete_empty_rows_values(data)                     
                
                

                for i in range(0,len(data.columns)):
                    for j in range(0,data.shape[0]):
                        try:
                            cell=(data[i][j])
                            if(pd.isnull(cell)) or cell=='\n':
                                data[i][j]=np.nan
                            if ord(cell)==8212 or ord(cell)==36:
                                data[i][j]=np.nan
                        except:
                            pass
                        
               
                data=correct_numbers_table(data)
                data=interpolate(data)


                # mattype=[]
                # for i in range(0,data.shape[0]):
                #     mattype.append("year")
                # data["mattype"]=mattype
                

                #convert entries to float
                data=data.astype(float)

                #format headings name

                # if newcolsdict[0]=="":
                #     newcolsdict[0]="Year"
                # x=newcolsdict.values()
                # data.columns=list(x)

                principalCount=0
                interestCount=0
                totalCount=0
                otherCount=0
                swapCount=0

                principalIndex=[]
                interestIndex=[]
                totalIndex=[]
                otherIndex=[]
                swapIndex=[]

                data=delete_empty_columns(data,headings)

                while("" in headings) :
                    headings.remove("")

                print(headings)

                for i in range(0,len(headings)):
                    head=str(headings[i])
                    head=head.lower()
                    if head=="" or head=="\n":
                        continue
                    elif "principal" in head:
                        principalCount+=1
                        principalIndex.append(i)
                    elif "interest" in head:
                        interestCount+=1
                        interestIndex.append(i)
                    elif "total" in head:
                        totalCount+=1
                        totalIndex.append(i)
                    elif "swap" in head or "net" in head:
                        swapCount+=1
                        swapIndex.append(i)
                    else:
                        otherCount+=1 
                        otherIndex.append(i) 

       
                columnCount=principalCount+interestCount+totalCount+otherCount+swapCount
                if(len(data.columns)==columnCount):
                    print("MATCHES")
                else:
                    extra=len(data.columns)-columnCount
                    data=delete_empty_columns_extra(data,extra)
                print(len(data.columns))
                print(principalCount,interestCount,swapCount,totalCount,otherCount)
                

                headings=format_headings(headings)
                print(headings)


                if(len(data.columns)==columnCount):
                    data.columns=headings
                    
                yearsCol=data.iloc[:, 0]

                principalInd=[]
                for i in range(0,len(headings)):
                    if "Principal" in headings[i]:
                        principalInd.append(i)

                print(principalInd)

                data=data.iloc[:,1:]

                if table_id=="id_p127_1":
                    col_list = list(data)
                    x=data.iloc[:,3]
                    y=data.iloc[:,4]
                    data.iloc[:,4]=x
                    data.iloc[:,3]=np.zeros(len(x))

                
                if len(principalInd)==1:
                    x=data.iloc[:,:principalInd[0]-1]
                    y=data.iloc[:,principalInd[0]-1:]
                    
                    if x.empty==False:
                        x.insert(loc=0, column='Due Year', value=yearsCol)
                        finalData.append(x)
                        print(x)
                    if y.empty==False:
                        y.insert(loc=0, column='Due Year', value=yearsCol)
                        finalData.append(y)
                        print(y)
                        

                else:                
                    for i in range(0,len(principalInd)-1):
                        a=principalInd[i]
                        b=principalInd[i+1]
                        x=data.iloc[:,:a-1]
                        y=data.iloc[:,a-1:b-1]
                        
                        if x.empty==False:
                            x.insert(loc=0, column='Due Year', value=yearsCol)
                            finalData.append(x)
                            print(x)
                        if y.empty==False:
                            y.insert(loc=0, column='Due Year', value=yearsCol)
                            finalData.append(y)
                            print(y)
                    z=data.iloc[:,principalInd[-1]-1:]
                    
                    if z.empty==False:
                        z.insert(loc=0, column='Due Year', value=yearsCol)
                        finalData.append(z)
                        print(z)
                
                print(finalData)